In [157]:
import pandas as pd

sep_symbols = [';', ',', '\t', '.', '/', '\\']

def read_auto_sep():
    success = False
    for sep in sep_symbols:
        try:
            df = pd.read_csv('soap.csv', sep=sep)
            if len(df.columns) < 2:
                raise ValueError("Skip")
        except:
            continue
        else:
            success = True
            break
    if success:
        return df
    else:
        raise ValueError("Can't detect separator")

df = read_auto_sep()
df_drop = df.drop_duplicates()
if df_drop.shape[0] > 1000:
    df_drop = df_drop.sample(1000)

In [177]:
chart_data = {}
for column, _type in zip(df.columns, df.dtypes):
    if _type == 'int':
        chart_data[column] = {'type': 'line', 'data': df[column]}
    elif _type == 'object':
        sep = get_separator(df_drop[column])
        is_cat = check_categories(df_drop[column])
        if is_cat and not sep:
            chart_data[column] = {'type': 'pie', 'data': {
                unique: amount / df.shape[0] for unique, amount in df[column].value_counts().items()
            }}
            continue
        if sep:
            chart_data[column] = {'type': 'bar', 'data': value_counts(df[column].str.split(sep)).to_dict()}

In [178]:
chart_data

{'Какими типами мыла вы пользуетесь на постоянной основе? ': {'type': 'bar',
  'data': {'Классическое гигиеническое': 1825,
   'Жидкое': 1827,
   'Ручной работы': 1769,
   'Хозяйственное': 1784,
   'Пилинговое/Скрабовое': 1765,
   'антибактериальное': 1,
   'Детское': 1766,
   'Мыло для бритья': 1759,
   'Дегтярное': 1}},
 'Какую марку мыла вы предпочитаете? ': {'type': 'bar',
  'data': {'Dove': 1799,
   'Synergetic': 1762,
   'LUSH': 1765,
   'Palmolive': 1780,
   'Чистая линия': 1769,
   'Мне все равно на бренд': 60,
   'Safeguard': 1765,
   'Бархатные ручки': 1764,
   'Рецепты бабушки Агафьи': 1764,
   'Bath and Body Works': 1,
   'Лента': 1,
   'Вiленскае барока ': 1,
   'Каждый день/красная цена': 1759,
   'Lusob': 1,
   "Johnson's": 1758,
   'иностранные эко-бренды': 1}},
 'На что вы смотрите при выборе мыла?': {'type': 'bar',
  'data': {'Цена': 1819,
   'Запах': 1829,
   'Тактильные ощущения': 1784,
   'Полезные свойства': 1788,
   'Внешний вид': 1785,
   'чтобы сложнее уронить 

In [171]:
for i, j in df['Какую цену для натурального мыла ручной работы вы считаете приемлемой? (150г)'].value_counts().items():
    print(i, j)

до 100 рублей 1770
от 100 до 200 рублей 44
от 200 до 300 рублей 31
свыше 300 рублей 10
хз 1
Не знаю 1
смотря какие ингредиенты входят в состав и какая у него форма, так что от 200 и выше 1


In [155]:
from collections import Counter

def check_bigtext(series):
    pass


def value_counts(column):
    c = Counter()
    for row in column:
        if isinstance(row, float):
            continue
        for name in row:
            c[name] += 1
    return pd.Series(c)

def check_categories(series):
    return len(series.unique()) <= 20
    
def get_separator(series):
    sep_symbols = ';,'
    scores = {}
    for symbol in sep_symbols:
        scores[symbol] = 0
        counted_symbol = series.str.count(symbol)
        if len(counted_symbol.unique()) < 2:
            continue

        if counted_symbol.max() != counted_symbol.min():
            scores[symbol] += 2
        vc = counted_symbol.value_counts()
        if (vc[counted_symbol.max()] / len(counted_symbol) < 0.1) or (vc[counted_symbol.min()] / len(counted_symbol) < 0.1):
            scores[symbol] -= 2

        if (vc[counted_symbol.max()] / len(counted_symbol) < 0.8) and \
           (vc[counted_symbol.min()] / len(counted_symbol) < 0.8) and (len(vc) > 2):
            scores[symbol] += 3
        #TODO Add more criterias
        
    res = list(filter(lambda pair: pair[1] > 0, scores.items()))
    return max(res, key=lambda x: x[1])[0] if len(res) > 0 else None 